## Final Project - Group
- Himanvesh Maddina


## PART 1

## Loading the data 

In [1]:
%reload_ext sql

In [2]:
%sql postgresql://postgres@localhost/postgres

In [3]:
!pwd

/home/ubuntu/notebooks


In [4]:
import os

!mkdir bulk

In [5]:
os.chdir("bulk")

bulk loading data

In [6]:
!wget -i cases.csv

--2021-12-07 00:30:19--  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-12-2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8014 (7.8K) [text/plain]
Saving to: ‘04-12-2020.csv’

04-12-2020.csv      100%[===================>]   7.83K  --.-KB/s    in 0s      

2021-12-07 00:30:19 (64.5 MB/s) - ‘04-12-2020.csv’ saved [8014/8014]

--2021-12-07 00:30:19--  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-13-2020.csv
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 9593 (9.4K) [text/plain]
Saving to: ‘04-13-2020.csv’

04-13-2020.csv      100%[=========

skacting files based on their year

!csvstack -d "," *2021.csv | csvcut > cm21.csv

In [8]:
!csvstack -d "," *2021.csv | csvcut -d ',' -c1:12,14:17 > cm21.csv

In [9]:
!csvstack -d "," *2020.csv | csvcut -d ',' -c1:12,14:17 > cm20.csv

In [10]:
!csvstack -d "," cm*.csv > cm.csv

!csvstack -d "," *2020.csv | csvcut -d ',' -c1,3,6:12,14,15,17 > cm20.csv

In [6]:
!csvcut -n cm.csv

  1: Province_State
  2: Country_Region
  3: Last_Update
  4: Lat
  5: Long_
  6: Confirmed
  7: Deaths
  8: Recovered
  9: Active
 10: FIPS
 11: Incident_Rate
 12: People_Tested
 13: Mortality_Rate
 14: UID
 15: ISO3
 16: Testing_Rate


In [7]:
!rm -f *21.csv

In [8]:
!rm -f *20.csv

## Spark

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
from pyspark import SparkContext

In [6]:
spark = SparkContext(appName='FinalProject')

21/12/09 22:55:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark

<SparkContext master=local[*] appName=FinalProject>

In [8]:
covid = spark.textFile('cm.csv')

In [9]:
covid.count()

34703

## PART 2

### DB Design

### Creating Table Cases

In [11]:
%%sql
DROP TABLE IF EXISTS Cases cascade;

CREATE TABLE Cases (
    PROVINCE_STATE VARCHAR(30) NOT NULL,
    Country_Region VARCHAR(2) NOT NULL,
    Last_Update DATE,
    Lat NUMERIC ,
    Long_ NUMERIC ,
    Confirmed INTEGER ,
    Deaths INTEGER,
    Recovered NUMERIC,
    Active NUMERIC,
    FIPS NUMERIC,
    Incident_Rate NUMERIC,
    Total_Test_Results NUMERIC,
    Case_Fatality_Ratio NUMERIC,
    UID NUMERIC,
    ISO3 VARCHAR(3),
    Testing_Rate NUMERIC
);

 * postgresql://postgres@localhost/postgres
Done.
Done.


[]

In [13]:
%%sql
COPY Cases FROM '/home/ubuntu/notebooks/cm.csv'
CSV
HEADER;

 * postgresql://postgres@localhost/postgres
34702 rows affected.


[]

In [14]:
%%sql
SELECT * FROM Cases order by last_update desc limit 5;

 * postgresql://postgres@localhost/postgres
5 rows affected.


province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,total_test_results,case_fatality_ratio,uid,iso3,testing_rate
Virgin Islands,US,None,18.3358,-64.8963,51,1,43,50,None,47.54446806,350,1.960784314,850,VIR,326.2855651
American Samoa,US,None,-14.270999999999999,-170.132,0,0,None,None,60,0.0,3,None,16,ASM,5.3917075537822825
American Samoa,US,None,-14.271,-170.1322,0,0,None,0,60,0.0,3,None,16,ASM,5.391707554
Recovered,US,None,None,None,0,0,32988,-32988,None,None,None,None,84070001,USA,None
Recovered,US,None,None,None,0,0,43482,-43482.0,None,None,None,None,84070001,USA,None


In [15]:
%%sql
SELECT * FROM Cases WHERE lat is null and long_ is null order by last_update limit 6;

 * postgresql://postgres@localhost/postgres
6 rows affected.


province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,total_test_results,case_fatality_ratio,uid,iso3,testing_rate
Grand Princess,US,2020-04-12,None,None,103,0,0,103,999,None,None,0.0,84099999,USA,None
Diamond Princess,US,2020-04-12,None,None,49,0,0,49,888,None,None,0.0,84088888,USA,None
Diamond Princess,US,2020-04-13,None,None,49,0,0,49.0,88888,None,None,0.0,84088888,USA,None
Grand Princess,US,2020-04-13,None,None,103,0,0,103.0,99999,None,None,0.0,84099999,USA,None
Diamond Princess,US,2020-04-14,None,None,49,0,0,49.0,88888,None,None,0.0,84088888,USA,None
Grand Princess,US,2020-04-14,None,None,103,0,0,103.0,99999,None,None,0.0,84099999,USA,None


In [16]:
%%sql
SELECT * FROM Cases WHERE fips is null order by last_update limit 5;

 * postgresql://postgres@localhost/postgres
5 rows affected.


province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,total_test_results,case_fatality_ratio,uid,iso3,testing_rate
Recovered,US,2020-04-21,None,None,0,0,75204,-75204.0,None,None,None,None,84070001,USA,None
Recovered,US,2020-04-22,None,None,0,0,77366,-77366.0,None,None,None,None,84070001,USA,None
Recovered,US,2020-04-24,None,None,0,0,80203,-80203.0,None,None,None,None,84070001,USA,None
Recovered,US,2020-04-25,None,None,0,0,99079,-99079.0,None,None,None,None,84070001,USA,None
Recovered,US,2020-04-26,None,None,0,0,100372,-100372.0,None,None,None,None,84070001,USA,None


#### Deleting rows where province_state is recovered

In [17]:
%%sql
delete from Cases where PROVINCE_STATE = 'Recovered' and fips is null;

 * postgresql://postgres@localhost/postgres
18 rows affected.


[]

#### Converting UID, ISo, active, fips and test results to INTEGER

In [18]:
%%sql
SELECT cast(UID as INTEGER) FROM Cases limit 5;

 * postgresql://postgres@localhost/postgres
5 rows affected.


uid
84000001
84000002
84000004
84000005
84000006


In [19]:
%%sql
ALTER TABLE Cases add column UID1 INTEGER;
UPDATE Cases set UID1 = cast(UID as INTEGER);
ALTER TABLE Cases DROP Column UID;

 * postgresql://postgres@localhost/postgres
Done.
34684 rows affected.
Done.


[]

In [20]:
%%sql
ALTER TABLE Cases add column FIPS1 INTEGER;
UPDATE Cases set FIPS1 = cast(FIPS as INTEGER);
ALTER TABLE Cases DROP Column FIPS;

 * postgresql://postgres@localhost/postgres
Done.
34684 rows affected.
Done.


[]

In [21]:
%%sql
ALTER TABLE Cases add column Total_Test_Results1 INTEGER;
UPDATE Cases set Total_Test_Results1 = cast(Total_Test_Results as INTEGER);
ALTER TABLE Cases DROP Column Total_Test_Results;

 * postgresql://postgres@localhost/postgres
Done.
34684 rows affected.
Done.


[]

In [22]:
%%sql
ALTER TABLE Cases add column Active1 INTEGER;
UPDATE Cases set Active1 = cast(Active as INTEGER);
ALTER TABLE Cases DROP Column Active;

 * postgresql://postgres@localhost/postgres
Done.
34684 rows affected.
Done.


[]

In [23]:
%%sql
ALTER TABLE Cases add column recovered1 INTEGER;
UPDATE Cases set recovered1 = cast(Recovered as INTEGER);
ALTER TABLE Cases DROP Column Recovered;

 * postgresql://postgres@localhost/postgres
Done.
34684 rows affected.
Done.


[]

#### Dropping columns of Grand Princess and Diamond Princess

In [24]:
%%sql
delete from Cases where PROVINCE_STATE = 'Grand Princess' or PROVINCE_STATE = 'Diamond Princess';

 * postgresql://postgres@localhost/postgres
1196 rows affected.


[]

In [25]:
%%sql
SELECT * FROM Cases limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


province_state,country_region,last_update,lat,long_,confirmed,deaths,incident_rate,case_fatality_ratio,iso3,testing_rate,uid1,fips1,total_test_results1,active1,recovered1
Alaska,US,2020-04-12,61.3707,-152.4044,272,8,45.50404936,2.941176471,USA,1344.711576,84000002,2,8038,264,66
Wyoming,US,2020-04-13,42.756,-107.3025,275,1,55.30528517416137,0.36363636363636365,USA,1199.4208028316307,84000056,56,5964,274,138
American Samoa,US,2020-08-20,-14.271,-170.132,0,0,0.0,None,ASM,2721.015078808792,16,60,1514,0,None
Alabama,US,2020-04-12,32.3182,-86.9023,3667,93,75.98802021,2.610159978,USA,1265.0756600046705,84000001,1,62029,3470,None
Arizona,US,2020-04-12,33.7298,-111.4312,3542,115,48.66242224,3.246753247,USA,578.5222863,84000004,4,52289,3427,None
Arkansas,US,2020-04-12,34.9697,-92.3731,1280,27,49.43942261,2.109375,USA,761.7533537,84000005,5,19722,1253,367
California,US,2020-04-12,36.1162,-119.6816,22182,631,58.13772584,2.81202018,USA,485.4238685,84000006,6,190328,22155,None
Colorado,US,2020-04-12,39.0598,-105.3111,7307,289,128.943729,3.955111537,USA,615.3899908,84000008,8,34873,7018,None
Connecticut,US,2020-04-12,41.5978,-72.7554,12035,554,337.5604825,4.603240548,USA,1156.148159,84000009,9,41220,11481,None
Delaware,US,2020-04-12,39.3185,-75.5071,1625,49,166.8782169,2.153846154,USA,1140.214672,84000010,10,11103,1590,191


#### Creating VaccineP Table

In [26]:
!csvcut -n people_vaccinated_us_timeline.csv

  1: FIPS
  2: Province_State
  3: Country_Region
  4: Date
  5: Lat
  6: Long_
  7: Combined_Key
  8: People_Fully_Vaccinated
  9: People_Partially_Vaccinated


In [27]:
%%sql
DROP TABLE IF EXISTS VaccineP cascade;

CREATE TABLE VaccineP (
    FIPS INTEGER,
    PROVINCE_STATE VARCHAR(50) NOT NULL,
    Country_Region VARCHAR(2) NOT NULL,
    Date DATE NOT NULL,
    Lat NUMERIC,
    Long_ NUMERIC,
    Combined_Key VARCHAR(50),
    People_Fully_Vaccinated INTEGER,
    People_Partially_Vaccinated INTEGER
)

 * postgresql://postgres@localhost/postgres
Done.
Done.


[]

In [29]:
%%sql
COPY VaccineP FROM '/home/ubuntu/notebooks/people_vaccinated_us_timeline.csv'
CSV
HEADER;

 * postgresql://postgres@localhost/postgres
22092 rows affected.


[]

In [30]:
%%sql
SELECT * FROM VaccineP order by date limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


fips,province_state,country_region,date,lat,long_,combined_key,people_fully_vaccinated,people_partially_vaccinated
9,Connecticut,US,2020-12-10,41.5978,-72.7554,"Connecticut, US",None,None
10,Delaware,US,2020-12-10,39.3185,-75.5071,"Delaware, US",None,None
8,Colorado,US,2020-12-10,39.0598,-105.3111,"Colorado, US",None,None
4,Arizona,US,2020-12-10,33.7298,-111.4312,"Arizona, US",None,None
5,Arkansas,US,2020-12-10,34.9697,-92.3731,"Arkansas, US",None,None
60,American Samoa,US,2020-12-10,-14.271,-170.132,"American Samoa, US",None,None
1,Alabama,US,2020-12-10,32.3182,-86.9023,"Alabama, US",None,None
6,California,US,2020-12-10,36.1162,-119.6816,"California, US",None,None
2,Alaska,US,2020-12-10,61.3707,-152.4044,"Alaska, US",None,None
None,Department of Defense,US,2020-12-10,None,None,"Department of Defense, US",None,None


In [31]:
%%sql
SELECT * FROM VaccineP WHERE fips is null order by date limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


fips,province_state,country_region,date,lat,long_,combined_key,people_fully_vaccinated,people_partially_vaccinated
None,Department of Defense,US,2020-12-10,None,None,"Department of Defense, US",None,None
None,Indian Health Services,US,2020-12-10,None,None,"Indian Health Services, US",None,None
None,Federal Bureau of Prisons,US,2020-12-10,None,None,"Federal Bureau of Prisons, US",None,None
None,Long Term Care (LTC) Program,US,2020-12-10,None,None,"Long Term Care (LTC) Program, US",None,None
None,Veterans Health Administration,US,2020-12-10,None,None,"Veterans Health Administration, US",None,None
None,Department of Defense,US,2020-12-11,None,None,"Department of Defense, US",None,None
None,Federal Bureau of Prisons,US,2020-12-11,None,None,"Federal Bureau of Prisons, US",None,None
None,Indian Health Services,US,2020-12-11,None,None,"Indian Health Services, US",None,None
None,Long Term Care (LTC) Program,US,2020-12-11,None,None,"Long Term Care (LTC) Program, US",None,None
None,Veterans Health Administration,US,2020-12-11,None,None,"Veterans Health Administration, US",None,None


In [32]:
%%sql 
ALTER Table VaccineP DROP column lat,
                    DROP column long_,
                    DROP column province_state,
                    DROP column country_region;

 * postgresql://postgres@localhost/postgres
Done.


[]

#### Assigning FIPS values for none values

In [33]:
%%sql
UPDATE VaccineP set fips = 90 where combined_key = 'Department of Defense, US';
UPDATE VaccineP set fips = 91 where combined_key = 'Long Term Care (LTC) Program, US';
UPDATE VaccineP set fips = 92 where combined_key = 'Indian Health Services, US';
UPDATE VaccineP set fips = 93 where combined_key = 'Federal Bureau of Prisons, US';
UPDATE VaccineP set fips = 94 where combined_key = 'Veterans Health Administration, US';

 * postgresql://postgres@localhost/postgres
363 rows affected.
312 rows affected.
363 rows affected.
363 rows affected.
363 rows affected.


[]

In [34]:
%%sql 
SELECT count(fips),fips FROM VaccineP group by fips order by fips desc limit 6;

 * postgresql://postgres@localhost/postgres
6 rows affected.


count,fips
363,94
363,93
363,92
312,91
363,90
363,78


#### Creating timeseries and vaccine Tables

In [35]:
!csvcut -n time_series_covid19_US.csv

  1: date
  2: state
  3: cases_conf_probable
  4: cases_confirmed
  5: cases_probable
  6: tests_viral_positive
  7: tests_viral_negative
  8: tests_viral_total
  9: tests_antigen_positive
 10: tests_antigen_total
 11: people_viral_positive
 12: people_viral_total
 13: people_antigen_positive
 14: people_antigen_total
 15: encounters_viral_total
 16: tests_combined_total


In [36]:
!csvcut -n vaccine_data_us_timeline.csv

  1: Province_State
  2: Date
  3: Vaccine_Type
  4: FIPS
  5: Country_Region
  6: Lat
  7: Long_
  8: Doses_alloc
  9: Doses_shipped
 10: Doses_admin
 11: Stage_One_Doses
 12: Stage_Two_Doses
 13: Combined_Key


In [37]:
%%sql
DROP TABLE IF EXISTS timeseries cascade;

CREATE TABLE timeseries (
    date DATE NOT NULL,
    state VARCHAR(2),
    cases_conf_probable INTEGER,
    cases_confirmed INTEGER,
    cases_probable INTEGER,
    tests_viral_positive INTEGER,
    tests_viral_negative INTEGER,
    tests_viral_total INTEGER,
    tests_antigen_positive INTEGER,
    tests_antigen_total INTEGER,
    people_viral_positive INTEGER,
    people_viral_total INTEGER,
    people_antigen_positive INTEGER,
    people_antigen_total INTEGER,
    encounters_viral_total INTEGER,
    tests_combined_total INTEGER
)

 * postgresql://postgres@localhost/postgres
Done.
Done.


[]

In [38]:
%%sql
COPY timeseries FROM '/home/ubuntu/notebooks/time_series_covid19_US.csv'
CSV
HEADER;

 * postgresql://postgres@localhost/postgres
36180 rows affected.


[]

In [39]:
%%sql
DROP TABLE IF EXISTS vaccine cascade;

CREATE TABLE vaccine (
    Province_State Varchar(30) NOT NULL,
    Date DATE NOT NULL,
    Vaccine_Type Varchar(20),
    FIPS INTEGER,
    Country_Region Varchar(2),
    Lat NUMERIC,
    Long_ NUMERIC,
    Doses_alloc INTEGER, 
    Doses_shipped INTEGER,
    Doses_admin INTEGER,
    Stage_One_Doses INTEGER,
    Stage_Two_Doses INTEGER,
    Combined_Key varchar(50)
)

 * postgresql://postgres@localhost/postgres
Done.
Done.


[]

In [41]:
%%sql
COPY vaccine FROM '/home/ubuntu/notebooks/vaccine_data_us_timeline.csv'
CSV
HEADER;

 * postgresql://postgres@localhost/postgres
120274 rows affected.


[]

In [42]:
%%sql
SELECT * FROM Vaccine order by date limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


province_state,date,vaccine_type,fips,country_region,lat,long_,doses_alloc,doses_shipped,doses_admin,stage_one_doses,stage_two_doses,combined_key
American Samoa,2020-12-10,All,60,US,-14.271,-170.132,None,None,None,None,None,"American Samoa, US"
American Samoa,2020-12-10,Pfizer,60,US,-14.271,-170.132,None,None,None,None,None,"American Samoa, US"
Alaska,2020-12-10,Pfizer,2,US,61.3707,-152.4044,None,None,None,None,None,"Alaska, US"
Alaska,2020-12-10,Moderna,2,US,61.3707,-152.4044,None,None,None,None,None,"Alaska, US"
Alabama,2020-12-10,Pfizer,1,US,32.3182,-86.9023,None,None,None,None,None,"Alabama, US"
American Samoa,2020-12-10,Moderna,60,US,-14.271,-170.132,None,None,None,None,None,"American Samoa, US"
Alabama,2020-12-10,Moderna,1,US,32.3182,-86.9023,None,None,None,None,None,"Alabama, US"
Alaska,2020-12-10,All,2,US,61.3707,-152.4044,None,None,None,None,None,"Alaska, US"
Alabama,2020-12-10,All,1,US,32.3182,-86.9023,None,None,None,None,None,"Alabama, US"
Arizona,2020-12-10,All,4,US,33.7298,-111.4312,None,None,None,None,None,"Arizona, US"


In [43]:
%%sql
UPDATE Vaccine set fips = 90 where province_state = 'Department of Defense';
UPDATE Vaccine set fips = 91 where province_state = 'Long Term Care (LTC) Program';
UPDATE Vaccine set fips = 92 where province_state = 'Indian Health Services';
UPDATE Vaccine set fips = 93 where province_state = 'Federal Bureau of Prisons';
UPDATE Vaccine set fips = 94 where province_state = 'Veterans Health Administration';
UPDATE Vaccine set fips = 95 where province_state = 'Bureau of Prisons';
UPDATE Vaccine set fips = 96 where province_state = 'Federated States of Micronesia';
UPDATE Vaccine set fips = 97 where province_state = 'Marshall Islands';
UPDATE Vaccine set fips = 98 where province_state = 'Republic of Palau';

 * postgresql://postgres@localhost/postgres
1913 rows affected.
1877 rows affected.
1913 rows affected.
1636 rows affected.
1913 rows affected.
309 rows affected.
307 rows affected.
307 rows affected.
307 rows affected.


[]

In [46]:
%%sql
DROP TABLE IF EXISTS state CASCADE;

CREATE TABLE state (
    FIPS INTEGER,
    province_state VARCHAR(64),
    latitude NUMERIC,
    longitude NUMERIC,
    country_region VARCHAR(3),
    UID INTEGER
);

 * postgresql://postgres@localhost/postgres
Done.
Done.


[]

In [47]:
%%sql
INSERT INTO state (FIPS, province_state, latitude, longitude, country_region)
SELECT DISTINCT FIPS,
        province_state,
        lat as latitude,
        long_ as longitude,
        country_region
FROM vaccine
GROUP BY FIPS, province_state, latitude, longitude, country_region
HAVING COUNT(*) > 1;

 * postgresql://postgres@localhost/postgres
68 rows affected.


[]

In [48]:
%%sql
SELECT * FROM state order by fips limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


fips,province_state,latitude,longitude,country_region,uid
1,Alabama,32.3182,-86.9023,US,None
2,Alaska,61.3707,-152.4044,US,None
4,Arizona,33.7298,-111.4312,US,None
5,Arkansas,34.9697,-92.3731,US,None
6,California,36.1162,-119.6816,US,None
8,Colorado,39.0598,-105.3111,US,None
9,Connecticut,41.5978,-72.7554,US,None
10,Delaware,39.3185,-75.5071,US,None
11,District of Columbia,38.8974,-77.0268,US,None
12,Florida,27.7663,-81.6868,US,None


In [49]:
%%sql 
delete from state where latitude = '44.571999999999996';
delete from state where longitude = '-99.78399999999999';
delete from state where latitude = '-14.270999999999999';

 * postgresql://postgres@localhost/postgres
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [50]:
%%sql 
Alter Table state 
ADD PRIMARY KEY (FIPS);

 * postgresql://postgres@localhost/postgres
Done.


[]

%%sql
ALTER TABLE c DROP CONSTRAINT cases_fips1_fkey; 

In [51]:
%%sql
ALTER TABLE Cases add FOREIGN KEY(fips1) references state (fips);

 * postgresql://postgres@localhost/postgres
Done.


[]

In [52]:
%%sql
ALTER TABLE Cases
DROP COLUMN lat,
DROP COLUMN long_,
DROP COLUMN province_state,
DROP COLUMN Country_region,
DROP COLUMN ISO3;

 * postgresql://postgres@localhost/postgres
Done.


[]

In [53]:
%%sql
ALTER TABLE Vaccine 
DROP COLUMN lat,
DROP COLUMN long_,
DROP COLUMN province_state,
DROP COLUMN Country_region,
DROP COLUMN Combined_key;

 * postgresql://postgres@localhost/postgres
Done.


[]

In [54]:
%%sql
ALTER TABLE VaccineP 
DROP COLUMN combined_key;

 * postgresql://postgres@localhost/postgres
Done.


[]

adding data to state

In [55]:
%%sql
DROP TABLE IF EXISTS data;

CREATE TABLE data (
    state_name VARCHAR(64),
    Population1 INTEGER
)

 * postgresql://postgres@localhost/postgres
Done.
Done.


[]

In [57]:
%%sql
COPY data FROM '/home/ubuntu/notebooks/state_demographics.csv'
CSV
HEADER;

 * postgresql://postgres@localhost/postgres
52 rows affected.


[]

In [58]:
%%sql 
ALTER TABLE state ADD Column population INTEGER;
UPDATE state
set 
population = population1
FROM data
WHERE state.province_state = data.state_name;

 * postgresql://postgres@localhost/postgres
Done.
52 rows affected.


[]

In [59]:
%%sql
DROP TABLE IF EXISTS data;

 * postgresql://postgres@localhost/postgres
Done.


[]

In [60]:
%%sql 
UPDATE state
set 
uid = uid1
FROM cases
WHERE state.fips = cases.fips1;

 * postgresql://postgres@localhost/postgres
56 rows affected.


[]

In [61]:
%%sql 
ALTER TABLE vaccine ADD Column fully_vaccinated INTEGER,
                    ADD COLUMN partially_vaccinated INTEGER;

 * postgresql://postgres@localhost/postgres
Done.


[]

In [62]:
%%sql
UPDATE vaccine
set 
fully_vaccinated = people_fully_vaccinated,
partially_vaccinated = people_partially_vaccinated
FROM vaccinep
WHERE vaccine.fips = vaccinep.fips and vaccine.date=vaccinep.date and (people_fully_vaccinated > 0 or people_partially_vaccinated >0 ) and vaccine_type ='All';

 * postgresql://postgres@localhost/postgres
19547 rows affected.


[]

In [63]:
%%sql
SElECT * FROM state order by fips limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


fips,province_state,latitude,longitude,country_region,uid,population
1,Alabama,32.3182,-86.9023,US,84000001,4903185
2,Alaska,61.3707,-152.4044,US,84000002,731545
4,Arizona,33.7298,-111.4312,US,84000004,7278717
5,Arkansas,34.9697,-92.3731,US,84000005,3017804
6,California,36.1162,-119.6816,US,84000006,39512223
8,Colorado,39.0598,-105.3111,US,84000008,5758736
9,Connecticut,41.5978,-72.7554,US,84000009,3565287
10,Delaware,39.3185,-75.5071,US,84000010,973764
11,District of Columbia,38.8974,-77.0268,US,84000011,705749
12,Florida,27.7663,-81.6868,US,84000012,21477737


In [64]:
%%sql
SElECT * FROM Cases limit 5;

 * postgresql://postgres@localhost/postgres
5 rows affected.


last_update,confirmed,deaths,incident_rate,case_fatality_ratio,testing_rate,uid1,fips1,total_test_results1,active1,recovered1
2020-04-12,272,8,45.50404936,2.941176471,1344.711576,84000002,2,8038,264,66
2020-04-13,275,1,55.30528517416137,0.36363636363636365,1199.4208028316307,84000056,56,5964,274,138
2020-08-20,0,0,0.0,None,2721.015078808792,16,60,1514,0,None
2020-04-12,3667,93,75.98802021,2.610159978,1265.0756600046705,84000001,1,62029,3470,None
2020-04-12,3542,115,48.66242224,3.246753247,578.5222863,84000004,4,52289,3427,None


In [65]:
%%sql
SElECT * FROM Vaccine limit 10;

 * postgresql://postgres@localhost/postgres
10 rows affected.


date,vaccine_type,fips,doses_alloc,doses_shipped,doses_admin,stage_one_doses,stage_two_doses,fully_vaccinated,partially_vaccinated
2020-12-10,All,1,None,None,None,None,None,None,None
2020-12-10,Moderna,1,None,None,None,None,None,None,None
2020-12-10,Pfizer,1,None,None,None,None,None,None,None
2020-12-11,All,1,None,None,None,None,None,None,None
2020-12-11,Moderna,1,None,None,None,None,None,None,None
2020-12-11,Pfizer,1,None,None,None,None,None,None,None
2020-12-12,All,1,None,None,None,None,None,None,None
2020-12-12,Moderna,1,None,None,None,None,None,None,None
2020-12-12,Pfizer,1,None,None,None,None,None,None,None
2020-12-13,All,1,None,None,None,None,None,None,None


In [66]:
%%sql
SElECT * FROM times limit 5;

 * postgresql://postgres@localhost/postgres
5 rows affected.


date,state,cases_conf_probable,cases_confirmed,cases_probable,tests_viral_positive,tests_viral_negative,tests_viral_total,tests_antigen_positive,tests_antigen_total,people_viral_positive,people_viral_total,people_antigen_positive,people_antigen_total,encounters_viral_total,tests_combined_total
2020-03-06,AK,None,None,None,None,None,8,None,None,None,None,None,None,None,8
2020-03-07,AK,None,None,None,None,None,12,None,None,None,None,None,None,None,12
2020-03-08,AK,None,None,None,None,None,14,None,None,None,None,None,None,None,14
2020-03-09,AK,None,None,None,None,None,23,None,None,None,None,None,None,None,23
2020-03-10,AK,None,None,None,None,None,23,None,None,None,None,None,None,None,23


In [67]:
%%sql
DROP TABLE IF EXISTS vaccineP;

 * postgresql://postgres@localhost/postgres
Done.


[]

In [69]:
#Q4

In [15]:
%%sql
SELECT * FROM state limit 10

 * postgresql://postgres@localhost/postgres
10 rows affected.


fips,province_state,latitude,longitude,country_region,uid,population
93,Federal Bureau of Prisons,None,None,US,None,None
90,Department of Defense,None,None,US,None,None
92,Indian Health Services,None,None,US,None,None
96,Federated States of Micronesia,None,None,US,None,None
94,Veterans Health Administration,None,None,US,None,None
97,Marshall Islands,None,None,US,None,None
91,Long Term Care (LTC) Program,None,None,US,None,None
98,Republic of Palau,None,None,US,None,None
95,Bureau of Prisons,None,None,US,None,None
2,Alaska,61.3707,-152.4044,US,84000002,731545


In [16]:
%%sql
SELECT * FROM cases limit 10

 * postgresql://postgres@localhost/postgres
10 rows affected.


last_update,confirmed,deaths,incident_rate,case_fatality_ratio,testing_rate,uid1,fips1,total_test_results1,active1,recovered1
2020-04-12,272,8,45.50404936,2.941176471,1344.711576,84000002,2,8038,264,66
2020-04-13,275,1,55.30528517416137,0.36363636363636365,1199.4208028316307,84000056,56,5964,274,138
2020-08-20,0,0,0.0,None,2721.015078808792,16,60,1514,0,None
2020-04-12,3667,93,75.98802021,2.610159978,1265.0756600046705,84000001,1,62029,3470,None
2020-04-12,3542,115,48.66242224,3.246753247,578.5222863,84000004,4,52289,3427,None
2020-04-12,1280,27,49.43942261,2.109375,761.7533537,84000005,5,19722,1253,367
2020-04-12,22182,631,58.13772584,2.81202018,485.4238685,84000006,6,190328,22155,None
2020-04-12,7307,289,128.943729,3.955111537,615.3899908,84000008,8,34873,7018,None
2020-04-12,12035,554,337.5604825,4.603240548,1156.148159,84000009,9,41220,11481,None
2020-04-12,1625,49,166.8782169,2.153846154,1140.214672,84000010,10,11103,1590,191


In [14]:
%%sql
SELECT fips,province_state FROM state WHERE fips < 57 Order by fips ASC

 * postgresql://postgres@localhost/postgres
51 rows affected.


fips,province_state
1,Alabama
2,Alaska
4,Arizona
5,Arkansas
6,California
8,Colorado
9,Connecticut
10,Delaware
11,District of Columbia
12,Florida


In [11]:
%%sql 
SELECT * 
FROM covid 
WHERE last_update = '2020-7-31'
ORDER BY incident_rate DESC 

 * postgresql://postgres@localhost/postgres
56 rows affected.


last_update,confirmed,deaths,incident_rate,case_fatality_ratio,testing_rate,fips1,total_test_results1,active1,recovered1
2020-07-31,114481,3925,2462.5956753514997,3.428516522392362,28338.317421679687,22,1317390,36310,74246
2020-07-31,170798,3626,2346.539919054416,2.122975678872118,12419.13925215117,4,1313170,144814,22358
2020-07-31,461379,6586,2148.17324562639,1.4274598540462395,27439.306105666532,12,5893342,454793,None
2020-07-31,414370,32516,2130.047038688701,7.887395323020489,29919.293439386238,36,5820368,308714,72973
2020-07-31,183205,15809,2037.4479717276934,8.735702050063546,23187.56973223946,34,2059564,132738,32423
2020-07-31,57579,1611,1934.6813617194568,2.7978950659094464,15116.21225953405,28,449881,20897,35071
2020-07-31,18950,1025,1788.8142002584577,5.313984168865435,19305.88345238309,44,204519,16130,1813
2020-07-31,86664,1565,1749.1079777736309,1.8248175182481752,21034.694795321815,1,1031370,48796,35401
2020-07-31,182286,3671,1716.8572826004954,2.0138683168208202,14517.543475474227,13,1541389,178615,None
2020-07-31,12057,584,1708.3977448072899,4.8436592850626194,25928.48165565945,11,182990,1696,9777


In [18]:
%%sql
SELECT * 
FROM cases 
WHERE last_update = '2020-7-31' AND fips1 < 57
ORDER BY incident_rate DESC 

 * postgresql://postgres@localhost/postgres
51 rows affected.


last_update,confirmed,deaths,incident_rate,case_fatality_ratio,testing_rate,uid1,fips1,total_test_results1,active1,recovered1
2020-07-31,114481,3925,2462.5956753514997,3.428516522392362,28338.317421679687,84000022,22,1317390,36310,74246
2020-07-31,170798,3626,2346.539919054416,2.122975678872118,12419.13925215117,84000004,4,1313170,144814,22358
2020-07-31,461379,6586,2148.17324562639,1.4274598540462395,27439.306105666532,84000012,12,5893342,454793,None
2020-07-31,414370,32516,2130.047038688701,7.887395323020489,29919.293439386238,84000036,36,5820368,308714,72973
2020-07-31,183205,15809,2037.4479717276934,8.735702050063546,23187.56973223946,84000034,34,2059564,132738,32423
2020-07-31,57579,1611,1934.6813617194568,2.7978950659094464,15116.21225953405,84000028,28,449881,20897,35071
2020-07-31,18950,1025,1788.8142002584577,5.313984168865435,19305.88345238309,84000044,44,204519,16130,1813
2020-07-31,86664,1565,1749.1079777736309,1.8248175182481752,21034.694795321815,84000001,1,1031370,48796,35401
2020-07-31,182286,3671,1716.8572826004954,2.0138683168208202,14517.543475474227,84000013,13,1541389,178615,None
2020-07-31,12057,584,1708.3977448072899,4.8436592850626194,25928.48165565945,84000011,11,182990,1696,9777


In [14]:
# %%sql 
# SELECT * 
# FROM covid 
# WHERE last_update = '2020-7-31'
# ORDER BY incident_rate DESC 

# only 1 to 56

In [26]:
%%sql
SELECT AVG(incident_rate)
FROM ( SELECT *
FROM cases 
WHERE last_update = '2020-7-31' AND fips1 < 57
ORDER BY incident_rate DESC Limit 10 ) as xyz

 * postgresql://postgres@localhost/postgres
1 rows affected.


avg
2002.2662417608030800


In [27]:
%%sql
SELECT AVG(incident_rate)
FROM ( SELECT *
FROM cases 
WHERE last_update = '2020-7-31' AND fips1 < 57
ORDER BY incident_rate ASC Limit 10 ) as xyz

 * postgresql://postgres@localhost/postgres
1 rows affected.


avg
379.4326608921969180


In [29]:
%%sql
SELECT AVG(incident_rate)
FROM ( SELECT *
FROM cases 
WHERE last_update = '2020-7-31' AND (fips1 = 1 OR fips1 = 56)
ORDER BY incident_rate DESC Limit 10 ) as xyz

 * postgresql://postgres@localhost/postgres
1 rows affected.


avg
1106.6022162145978200
